In [0]:
!pip3 install -q torch torchvision
!pip install --no-cache-dir -I pillow

    100% |████████████████████████████████| 2.0MB 58.7MB/s 


In [0]:
# based on https://github.com/zeruniverse/neural-colorization
import torch
import torch.nn as nn
import argparse
from torch.autograd import Variable
import torchvision.models as models
import torch.nn.functional as F
import os
from torch.utils import data
import numpy as np
from PIL import Image
from skimage.color import rgb2yuv,yuv2rgb
from scipy.ndimage import zoom
import cv2
from functools import reduce

import math

In [0]:
SIZE = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [0]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
base_dir = '/content/gdrive/My Drive/ColorMaskGAN'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class ColorGenerator(nn.Module):
    '''Generator'''
    def __init__(self):
        super(ColorGenerator, self).__init__()

        self.conv0_1 = nn.Conv2d(1,16, 3, stride=1, padding=1, bias=False)
        self.conv0_2 = nn.Conv2d(1,16, 3, stride=1, padding=1, bias=False)
        self.bn0 = nn.BatchNorm2d(16)
        self.relu0 = nn.LeakyReLU(0.1)
        
# 1 -> 32
        self.conv1 = nn.Conv2d(32, 64, 3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.LeakyReLU(0.1)

        self.conv2 = nn.Conv2d(64, 128, 3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.LeakyReLU(0.1)

        self.conv3 = nn.Conv2d(128, 256, 3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.LeakyReLU(0.1)

        self.conv4 = nn.Conv2d(256, 512, 3, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.LeakyReLU(0.1)

        self.conv5 = nn.Conv2d(512, 512, 3, stride=2, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.LeakyReLU(0.1)

        self.deconv6 = nn.ConvTranspose2d(512, 512, 3, stride=2, padding=1, output_padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512)
        self.relu6 = nn.ReLU()

        self.deconv7 = nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1, bias=False)
        self.bn7 = nn.BatchNorm2d(256)
        self.relu7 = nn.ReLU()

        self.deconv8 = nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1, bias=False)
        self.bn8 = nn.BatchNorm2d(128)
        self.relu8 = nn.ReLU()

        self.deconv9 = nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1, bias=False)
        self.bn9 = nn.BatchNorm2d(64)
        self.relu9 = nn.ReLU()
#Modifiquei para gerar só UV
        self.deconv10 = nn.ConvTranspose2d(64, 2, 3, stride=2, padding=1, output_padding=1, bias=False)
        self.bn10 = nn.BatchNorm2d(2)
        self.relu10 = nn.ReLU()

        self._initialize_weights()

    def forward(self, x, y):
      
        x = self.conv0_1(x)
        x = self.bn0(x)
        x = self.relu0(x)
        
        y = self.conv0_2(y)
        y = self.bn0(y)
        y = self.relu0(y)

#        print(x.shape, y.shape)
        x = torch.cat((x, y), 1)
      
        h = x
        h = self.conv1(h)
        h = self.bn1(h)
        h = self.relu1(h) # 64,112,112 (if input is 224x224)
        pool1 = h

        h = self.conv2(h)
        h = self.bn2(h)
        h = self.relu2(h) # 128,56,56
        pool2 = h

        h = self.conv3(h) # 256,28,28
        h = self.bn3(h)
        h = self.relu3(h)
        pool3 =h

        h = self.conv4(h) # 512,14,14
        h = self.bn4(h)
        h = self.relu4(h)
        pool4 = h

        h = self.conv5(h) # 512,7,7
        h = self.bn5(h)
        h = self.relu5(h)

        h = self.deconv6(h)
        h = self.bn6(h)
        h = self.relu6(h) # 512,14,14
        h += pool4

        h = self.deconv7(h)
        h = self.bn7(h)
        h = self.relu7(h) # 256,28,28
        h += pool3

        h = self.deconv8(h)
        h = self.bn8(h)
        h = self.relu8(h) # 128,56,56
        h += pool2

        h = self.deconv9(h)
        h = self.bn9(h)
        h = self.relu9(h) # 64,112,112
        h += pool1

        h = self.deconv10(h)
        h = F.tanh(h) # 3,224,224

        return h

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            if isinstance(m, nn.ConvTranspose2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

class ColorDiscriminator(nn.Module):
    '''Discriminator'''
    def __init__(self, large=False):
        super(ColorDiscriminator, self).__init__()
        
#         self.conv0_1 = nn.Conv2d(1,16, 3, stride=1, padding=1, bias=False)
#         self.conv0_2 = nn.Conv2d(1,16, 3, stride=1, padding=1, bias=False)
#         self.bn0 = nn.BatchNorm2d(16)
#         self.relu0 = nn.LeakyReLU(0.1)
#CHANGE CHANNELS        
        self.conv1 = nn.Conv2d(4, 64, 3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.LeakyReLU(0.1)

        self.conv2 = nn.Conv2d(64, 128, 3, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.LeakyReLU(0.1)

        self.conv3 = nn.Conv2d(128, 256, 3, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.LeakyReLU(0.1)

        self.conv4 = nn.Conv2d(256, 512, 3, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.LeakyReLU(0.1)

        self.conv5 = nn.Conv2d(512, 512, 3, stride=2, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.LeakyReLU(0.1)

        if large:
            self.conv6 = nn.Conv2d(512, 512, 15, stride=1, padding=0, bias=False)
        else:
            self.conv6 = nn.Conv2d(512, 512, 7, stride=1, padding=0, bias=False)
        self.bn6 = nn.BatchNorm2d(512)
        self.relu6 = nn.LeakyReLU(0.1)

        self.conv7 = nn.Conv2d(512, 1, 1, stride=1, padding=0, bias=False)

        self._initialize_weights()

    def forward(self, x, y):
      
#         x = self.conv0_1(x)
#         x = self.bn0(x)
#         x = self.relu0(x)
        
#         y = self.conv0_2(y)
#         y = self.bn0(y)
#         y = self.relu0(y)
        
#         x = torch.cat((x, y), 1)
        
#        h = x
        h = torch.cat((x, y), 1)
        h = self.conv1(h)
        h = self.bn1(h)
        h = self.relu1(h) # 64,112,112 (if input is 224x224)

        h = self.conv2(h)
        h = self.bn2(h)
        h = self.relu2(h) # 128,56,56

        h = self.conv3(h) # 256,28,28
        h = self.bn3(h)
        h = self.relu3(h)

        h = self.conv4(h) # 512,14,14
        h = self.bn4(h)
        h = self.relu4(h)

        h = self.conv5(h) # 512,7,7
        h = self.bn5(h)
        h = self.relu5(h)

        h = self.conv6(h)
        h = self.bn6(h)
        h = self.relu6(h) # 512,1,1

        h = self.conv7(h)
        h = F.sigmoid(h)

        return h

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            if isinstance(m, nn.ConvTranspose2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

In [0]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = {'training_dir':os.path.join(base_dir, "train"),
        'checkpoint_location': os.path.join(base_dir, "checkpoints"),
        'epoch': 120,
        'gpu': 0,
        'batch_size':12,
        'num_workers':6,
        'pixel_loss_weights':1000,
        "g_every":1,
        "g_lr":1e-4,
        "d_lr":1e-4,
        "checkpoint_every":1000,
        "d_init":os.path.join(base_dir, "D0.pth"),
        "g_init":os.path.join(base_dir, "G0.pth"),
        "model": os.path.join(base_dir, "G0.pth"),
        "test_image": os.path.join(base_dir, "test.jpg")
       }

args = dotdict(args)

# define data generator
class img_data(data.Dataset):
    def __init__(self, path):
        files = os.listdir(path)
        self.files = [os.path.join(path,x) for x in files]
    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img = Image.open(self.files[index])
        yuv = rgb2yuv(img)
        y = (yuv[...,0])-0.5
        u_t = yuv[...,1] / 0.43601035
        v_t = yuv[...,2] / 0.61497538
        return torch.Tensor(np.expand_dims(y,axis=0)),torch.Tensor(np.stack([u_t,v_t],axis=0))

In [0]:
# mask = Image.open(os.path.join(base_dir, 'mask.jpg'))
# mask = Variable(torch.Tensor(rgb2yuv(mask)[...,0]).unsqueeze(0).unsqueeze(0)).to(device)
mask_loss_weight = 100000

In [0]:
if not os.path.exists(os.path.join(args.checkpoint_location,'weights')):
    os.makedirs(os.path.join(args.checkpoint_location,'weights'))

# Define G, same as torch version
#G = generator().to(device)
# define D
#D = models.resnet18(pretrained=False,num_classes=2)
#D.fc = nn.Sequential(nn.Linear(2048, 1), nn.Sigmoid())
#D = D.to(device)

G = ColorGenerator().to(device)
D = ColorDiscriminator().to(device)

trainset = img_data(args.training_dir)
maskset = img_data(os.path.join(base_dir, "masks"))
params = {'batch_size': args.batch_size,
          'shuffle': True,
          'num_workers': args.num_workers}
training_generator = data.DataLoader(trainset, **params)
masks_generator = data.DataLoader(maskset, batch_size=1, shuffle=True)
#mask = mask.expand(args.batch_size, -1, -1, -1)

if args.test_image is not None:
    test_img = Image.open(args.test_image).convert('RGB').resize((SIZE,SIZE))
    test_yuv = rgb2yuv(test_img)
    test_inf = test_yuv[...,0].reshape(1,1,SIZE,SIZE)
    test_var = Variable(torch.Tensor(test_inf-0.5)).to(device)
if args.d_init is not None:
    D.load_state_dict(torch.load(args.d_init, map_location='cuda:0'))
if args.g_init is not None:
    G.load_state_dict(torch.load(args.g_init, map_location='cuda:0'))

# save test image for beginning

i=0
adversarial_loss = torch.nn.BCELoss()
optimizer_G = torch.optim.Adam(G.parameters(), lr=args.g_lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=args.d_lr, betas=(0.5, 0.999))
for epoch in range(0, args.epoch):
    for mask, _ in masks_generator:
      mask = mask + 0.5
      mask = mask.to(device)
      for y, uv in training_generator:
          # Adversarial ground truths
          valid = Variable(torch.Tensor(y.size(0), 1).fill_(1.0), requires_grad=False).to(device)
          fake = Variable(torch.Tensor(y.size(0), 1).fill_(0.0), requires_grad=False).to(device)

          yvar = Variable(y).to(device)
          uvvar = Variable(uv).to(device)
          real_imgs = torch.cat([yvar,uvvar],dim=1)


          mask20 = mask.expand(yvar.shape[0], -1, -1, -1)
          optimizer_G.zero_grad()
          uvgen = G(yvar, mask20) # MODIFIED HERE
          # Generate a batch of images
          gen_imgs = torch.cat([yvar.detach(),uvgen],dim=1)

          # Loss measures generator's ability to fool the discriminator
  #        print(gen_imgs.shape, mask20.shape, valid.shape)
#           HOP = .view(-1, 1)
          g_loss_gan = adversarial_loss(D(gen_imgs, mask20), valid)
          g_loss = (g_loss_gan 
                    + args.pixel_loss_weights * torch.mean((uvvar*mask20-uvgen)**2) 
#                     + mask_loss_weight * torch.mean(torch.mul(uvvar, mask20)**2)
                   )
          if i%args.g_every==0:
              g_loss.backward()
              optimizer_G.step()

          optimizer_D.zero_grad()

          # Measure discriminator's ability to classify real from generated samples
          real_loss = adversarial_loss(D(real_imgs, mask20), valid)
          fake_loss = adversarial_loss(D(gen_imgs.detach(), mask20.detach()), fake)
          d_loss = (real_loss + fake_loss) / 2
          d_loss.backward()
          optimizer_D.step()
          i+=1
          if i%args.checkpoint_every==0:
              print ("Epoch: %d: [D loss: %f] [G total loss: %f] [G GAN Loss: %f]" % (epoch, d_loss.item(), g_loss.item(), g_loss_gan.item()))

              torch.save(D.state_dict(), os.path.join(args.checkpoint_location,'weights','D'+str(epoch)+'.pth'))
              torch.save(G.state_dict(), os.path.join(args.checkpoint_location,'weights','G'+str(epoch)+'.pth'))
              if args.test_image is not None:
                  test_res = G(test_var, mask)
                  uv=test_res.cpu().detach().numpy()
                  uv[:,0,:,:] *= 0.436
                  uv[:,1,:,:] *= 0.615
                  test_yuv = np.concatenate([test_inf,uv],axis=1).reshape(3,SIZE,SIZE)
                  test_rgb = yuv2rgb(test_yuv.transpose(1,2,0))
                  cv2.imwrite(os.path.join(args.checkpoint_location,'test_epoch_'+str(epoch)+'.jpg'),(test_rgb.clip(min=0,max=1)*SIZE)[:,:,[2,1,0]])
torch.save(D.state_dict(), os.path.join(args.checkpoint_location,'D_final.pth'))
torch.save(G.state_dict(), os.path.join(args.checkpoint_location,'G_final.pth'))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([12, 1])) that is different to the input size (torch.Size([12, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4, 1, 1, 1])) is deprecated. Please ensure they have the same siz

Epoch: 0: [D loss: 0.790424] [G total loss: 18.751749] [G GAN Loss: 1.127248]
Epoch: 1: [D loss: 0.389419] [G total loss: 17.068918] [G GAN Loss: 1.922459]
Epoch: 2: [D loss: 0.084100] [G total loss: 14.632328] [G GAN Loss: 2.465448]
Epoch: 3: [D loss: 0.276594] [G total loss: 14.659258] [G GAN Loss: 3.302816]
Epoch: 4: [D loss: 0.006551] [G total loss: 17.098385] [G GAN Loss: 5.177050]
Epoch: 5: [D loss: 0.044293] [G total loss: 19.839882] [G GAN Loss: 3.810984]
Epoch: 6: [D loss: 0.012644] [G total loss: 20.432430] [G GAN Loss: 4.500637]
Epoch: 7: [D loss: 0.039813] [G total loss: 17.535648] [G GAN Loss: 3.933465]
Epoch: 7: [D loss: 0.004385] [G total loss: 17.779980] [G GAN Loss: 5.553062]
Epoch: 8: [D loss: 0.003467] [G total loss: 23.714794] [G GAN Loss: 6.177885]
Epoch: 9: [D loss: 0.093320] [G total loss: 25.188532] [G GAN Loss: 5.624502]
Epoch: 10: [D loss: 0.012877] [G total loss: 20.543989] [G GAN Loss: 4.873987]
Epoch: 11: [D loss: 0.069864] [G total loss: 20.757734] [G GAN 

In [0]:
args.input = os.path.join(base_dir, "images/sunflower.jpg")
args.output = os.path.join(base_dir, "color_sunflower.jpg")

#G = generator().to(device)
G = ColorGenerator().to(device)

if torch.cuda.is_available():    
    G.load_state_dict(torch.load(args.model, map_location='cuda:0'))
else:
    G.load_state_dict(torch.load(args.model,map_location='cpu'))

def inference(G,in_path,out_path):
    p=Image.open(in_path).convert('RGB')
    img_yuv = rgb2yuv(p)
    H,W,_ = img_yuv.shape
    infimg = np.expand_dims(np.expand_dims(img_yuv[...,0], axis=0), axis=0)
    img_variable = Variable(torch.Tensor(infimg-0.5))
    if args.gpu>=0:
        img_variable=img_variable.to(device)
    res = G(img_variable, mask)
    uv=res.cpu().detach().numpy()
    uv[:,0,:,:] *= 0.436
    uv[:,1,:,:] *= 0.615
    (_,_,H1,W1) = uv.shape
    uv = zoom(uv,(1,1,H/H1,W/W1))
    yuv = np.concatenate([infimg,uv],axis=1)[0]
    rgb=yuv2rgb(yuv.transpose(1,2,0))
    cv2.imwrite(out_path,(rgb.clip(min=0,max=1)*256)[:,:,[2,1,0]])


if not os.path.isdir(args.input):
    inference(G,args.input,args.output)
else:
    if not os.path.exists(args.output):
        os.makedirs(args.output)
    for f in os.listdir(args.input):
        inference(G,os.path.join(args.input,f),os.path.join(args.output,f))

In [0]:
if args.test_image is not None:
    test_res = G(test_var, mask)
    uv=test_res.cpu().detach().numpy()
    uv[:,0,:,:] *= 0.436
    uv[:,1,:,:] *= 0.615
    #print(test_var.shape, mask.shape, test_res.shape, uv.shape, test_inf.shape)
    test_yuv = np.concatenate([test_inf,uv],axis=1).reshape(3,SIZE,SIZE)
    test_rgb = yuv2rgb(test_yuv.transpose(1,2,0))
    cv2.imwrite(os.path.join(args.checkpoint_location,'test_init.jpg'),(test_rgb.clip(min=0,max=1)*SIZE)[:,:,[2,1,0]])

